# `CLOUD.jl` - 2D linear advection example

In [12]:
using OrdinaryDiffEq
using CLOUD;

Define the physical problem

In [13]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period
initial_data = InitialDataSine(A,k)
conservation_law = linear_advection_equation((a*cos(θ),a*sin(θ)),λ=1.0);

Set discretization parameters

In [14]:
M = 4
p = 4
p_map = 4
form = SplitConservationForm()
integrator = CarpenterKennedy2N54()
dt_scale=0.05
dt = dt_scale*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [15]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

visualize(spatial_discretization, results_path, "mesh.pdf", 
    plot_volume_nodes=false, grid_lines=true)

Run the solver and plot the solution

In [16]:
ode_problem = semidiscretize(load_project(results_path)...)
save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, integrator, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")
plotter = Plotter(spatial_discretization, results_path)
visualize(last(sol.u), plotter, "approx.pdf", u_range=[-1.0,1.0], contours=25,
    label="U^h(\\mathbf{x},t)")


 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:          151μs / 1573101.5%      11.9KiB / 21517358.2%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          81.5k    1.14s   48.1%  14.0μs    774MiB   31.1%  9.72KiB
   volume terms         81.5k    730ms   30.7%  8.95μs    500MiB   20.1%  6.28KiB
   facet terms          81.5k    187ms    7.9%  2.29μs    173MiB    6.9%  2.17KiB
   mass matrix solve    81.5k   92.8ms    3.9%  1.14μs   39.8MiB    1.6%     512B
 eval flux diff         81.5k    769ms   32.4%  9.44μs   0.99GiB   40.7%  12.7KiB
 extrapolate solution   81.5k    169ms    7.1%  2.07μs    142MiB    5.7%  1.78KiB
 eval numerical

In [17]:
visualize(initial_data, plotter, "exact.pdf", u_range=[-1.0,1.0], contours=17, label="U^0(\\mathbf{x})")

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [18]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.000683954884588324
Conservation (initial/final/diff):
[-1.108594540819019e-8][-1.1085945919933615e-8][-5.117434254131581e-16]
Energy (initial/final/diff):
[0.12499998811286066][0.12497956180878655][-2.0426304074103174e-5]


Set up a `DGMulti` scheme on triangles of degree `p` with `M` edges in each direction

In [19]:
reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), mapping_degree=p_map, N_plot=10)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgmulti/", overwrite=true, clear=true)

visualize(spatial_discretization, results_path, "mesh.pdf", 
    plot_volume_nodes=true, grid_lines=false)

Run the solver and plot the solution

In [20]:
ode_problem = semidiscretize(load_project(results_path)...)

save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, integrator, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

plotter = Plotter(spatial_discretization, results_path)
visualize(last(sol.u), plotter, "approx.pdf", u_range=[-1.0,1.0], contours=25,
    label="U^h(\\mathbf{x},t)")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:         82.9μs / 5366191.0%      11.9KiB / 43636281.0%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual           163k    2.11s   47.4%  12.9μs   2.40GiB   48.6%  15.4KiB
   volume terms          163k    1.44s   32.5%  8.86μs   1.92GiB   38.8%  12.3KiB
   mass matrix solve     163k    394ms    8.9%  2.42μs    371MiB    7.3%  2.33KiB
   facet terms           163k   64.4ms    1.4%   395ns   27.4MiB    0.5%     176B
 eval flux diff          163k    1.49s   33.4%  9.11μs   1.33GiB   26.9%  8.55KiB
 eval flux               163k    281ms    6.3%  1.72μs    139MiB    2.8%     896B
 gather externa

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [21]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0034367122388651375
Conservation (initial/final/diff):
[-1.4530229013823975e-7][-1.4530229025620095e-7][-1.1796119636642288e-16]
Energy (initial/final/diff):
[0.12499475017905026][0.12485504074741147][-0.0001397094316387959]
